# Asynchronous inference
In this notebook, we will configure a model deployment for asynchronous inference.
This notebook assumes that you are familiar with model deployments for local inference (if not, please have a look at [notebook 008](008_deploy_project.ipynb) first). 